# ProvaH with DeepFashion2

DeepFashion2 is a comprehensive fashion dataset. It contains 491K diverse images of 13 popular clothing categories from both commercial shopping stores and consumers. It totally has 801K clothing clothing items, where each item in an image is labeled with scale, occlusion, zoom-in, viewpoint, category, style, bounding box, dense landmarks(e.g 39 that means one of class and 38 of x,y position) and per-pixel mask.There are also 873K Commercial-Consumer clothes pairs.


<img src="images/deepfashion2_bigbang.png" width="800px" height="400px">

It is a solve benchmark of four tasks including clothes detection, pose estimation, segmentation, and retrieval, and strong baseline  is  proposed,  called  Match  R-CNN, which builds upon Mask R-CNN to solve these tasks in an end-to-end manner.

## Features of the deepFashon2

Each image has minimum single item while maximum 7 items. Each item is manually labeled with bounding box,  mask,  dense landmarks.
**So The ful processess will be clothes detection and recognition, landmark and pose estimation,  segmentation,  as well as verification and retrieval.**

<img src="images/multiple_item.png" width="600px" height="600px">

DeepFashion2 has four unique characteristics compared to existing fashion datasets.

- Large Sample Size.It contains 491K images of 43.8K clothing identities of interest.
- On average, each identity has 12.7 items with different styles such as color and printing.
- DeepFashion2 contained 801K items in total. 
- It is the largest fashion database to date.
- Furthermore, each item is associated with various annotations as introduced above.
- Versatility of DeepFashion2:
 - developed for multiple tasks of fashion understanding. 
 - Its rich annotations support clothes detection and classification,  dense  landmark  and pose estimation, instance segmentation, and cross-domaininstance-level clothes retrieval.
- Expressivity of DeepFashion2:
 - This is mainly reflected in two aspects:
   - First,  multiple items are present in a single image. 
   - Second, we have 13 different definitions of landmarks and poses for 13 different categories.
   
### some of these dense landmarks annotations:

<table><tr>
<td>  <img src="images/anotate1.png" width="300px" height="400px"> </td>
<td>  <img src="images/anotate2.png" width="300px" height="400px"> </td>
</tr></table>


### Back To meaning of data contain 801K item but 43.8K clothing identities

Actually some of images looks like exactly the same but different style of store representation is different from customer wear or style or color.

<td>  <img src="images/looks_like1.png" width="600px" height="600px"> </td>

Furthermore,  the items in first three columns and  second three columns from the same clothing identity but from two different domains, that is, commercial and customer.The items of the same identity may have different styles such as color and printing.

### Data Labeling
Category and Bounding Box.Human annotators area sked to draw a bounding box and assign a category label for each clothing item.There are 13 popular categories without ambiguity.and statistics per each category as in image below.

<td>  <img src="images/num_per_cat.png" width="600px" height="600px"> </td>


### Landmarks

23 defined landmarks for each category on average. Some definitions are shown in image of "Short sleeve top" above.These representations are different from human pose and style of wearing. They  facilitate  learning  of strong clothes features that satisfy realworld requirements and need.

Clothes Landmark, Contour, and Skeleton of different categories of clothes (e.g. upper-and-lower-body-garment) have different deformations and appearance changes,So they represent each category by defining its pose, which is aset of landmarks as well as contours and skeletons between landmarks.

Moreover,each landmark is assigned one of the two modes, **visible** or *occluded**.then generate contours and skeletons automatically by connecting landmarks in a certain order.To facilitate this process, annotators are also asked to distinguish landmarks into two types, contour point or junction point.The former one refers to keypoints at the boundary of an item,  while the latter one is assigned to keypoints in conjunctione e.g  **endpoint of strap on sling**.  The above process controls the labeling quality,because the generated skeletons help the annotators reexamine whether the landmarks are labeled with good quality.

### Mask

there are label per-pixel mask for each item in a semiautomatic manner with two stages. 
- The first stage automatically generates masks from the contours. 
- In the second stage, human annotators are asked to refine the masks,because the generated masks may be not accurate when complex human poses are presente.

### Benchmarks

they build four benchmarks by using the images and labels from  DeepFashion2.   For  each  benchmark,  there  are 391K images for training,  34K images for validation and 67K images for test which equal to the all images we have which are 491K.

### Four Task  solved

**Clothes Detection:** this task detects clothes in an image by predicting bounding boxes and category labels of the image belong to using the idea of Intersection Over Union (IOU), because some box maybe overlaps each other or pre overlaps.

**Landmark Estimation:** this task aims to predict landmarks for each detected clothing item in an each image.

So The idea of above is one image maybe have different clothesis items like your photo with your friend, so start with detect all of the clothes in the image as object detection then start to label which category this image belong to and start in process of predicting landmarks of each item we detect in the image.

**Segmentation:** this task assigned a category label(including background label) to each pixel in an item.

**Commercial-Consumer-Clothes-Retrieval:** given  a detected item from a consumer taken photo, this task aims to search the commercial images in the gallery for the items that are corresponding to this detected item.



### Model Process

<td>  <img src="images/matchRcnn.png" width="600px" height="400px"> </td>



The baseline model built upon Mask R-CNN, which is called  Match R-CNN, its an end-to-end training framework that jointly learns clothes detection, landmark estimation, instance segmentation, and consumer-to-shop retrieval.

Match R-CNN employs two images I1 and I2 as inputs, Each image is passed through three main components including:
 - Feature Network (FN)
 - Perceptron Network (PN)
 - Matching Network (MN)
 
#### Feature Network (FN)
FN contains a ResNet-FPN backbone,a region proposal network (RPN)  and RoIAlign module.

An image is first fed into ResNet50 to extract features,which are then fed into a FPN that uses a top-down architecture with lateral connections to build a pyramid of feature maps.RoIAlign extracts features from different levels of the pyramid map.
 
#### Perceptron Network (PN)

In the second stage,PN contains three streams of networks including landmark estimation, clothes detection,and mask prediction.

The extracted RoI features after the first stage are fed into three streams in PN separately:
 - the clothes detection stream has two hidden fully-connected (fc) layers,one fc layer for classification,and one fc layer for bounding box regression.
 - The stream of landmark estimation has 8 **conv** layers and 2 **deconv** layers to predict landmarks.
 - Segmentation stream has 4 **conv** layers, 1 **deconv**  layer, and another **conv** layer to predict masks.
 
#### Matching Network (MN)

In the third stage, MN contains a feature extractor and a similarity learning network for clothes retrieval.

The learned RoI features after the FN component are highly discriminative with respect to clothes category,  pose,  and mask, they are fed into MN to obtain features vectorsfor retrieval, where v1 and v2 are passed into the similarity learning network to obtain the similarity score between the detected clothing items in I1 and I2.

The similarity learning network consists of subtraction and square operator and a fc layer, which estimates the probability of whether two clothing items match or not.


## Loss Functions

**The parameters Θ of the Match R-CNN are optimized by minimizing five loss functions:**

 - cross-entrop L_clos for clothes classification
 - cross-entrop L_box for bounding box regression
 - cross-entrop L_pos  for landmark estimation
 - cross-entrop L_msk for clothes segmentation
 - cross-entrop L_pair for clothes retrieva

## Model Achievement

<img src="images/accuracy_1.png" style="width: 1000px; height=1000px"/>